## Installing PySpark in Jupyter Notebook and other environments.

In [15]:


%pip install findspark
%pip install pyspark
%pip install py4j



IOStream.flush timed out
Note: you may need to restart the kernel to use updated packages.


INFO:py4j.clientserver:Error while sending or receiving.
Traceback (most recent call last):
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/py4j/clientserver.py", line 503, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [Errno 104] Connection reset by peer
INFO:py4j.clientserver:Closing down clientserver connection
INFO:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/py4j/clientserver.py", line 503, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/

IOStream.flush timed out
Note: you may need to restart the kernel to use updated packages.
IOStream.flush timed out
Note: you may need to restart the kernel to use updated packages.


### Import all dependencies for the Spark job

In [16]:

# Importing necessary libraries

import findspark
import logging
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, count, max, min, collect_list
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.functions import year
from pyspark.sql.functions import col
from datetime import datetime




In [17]:
findspark.init()

In [18]:
# Set up logging
logging.basicConfig(level=logging.INFO)

# Initialize SparkSession
spark = SparkSession.builder. \
    appName("data_engineering_task"). \
    enableHiveSupport(). \
    getOrCreate()

#Reading data and printing schema values

In [19]:
# Load the dataset
data_file = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", ",") \
    .load("/workspaces/data_engineering_task/dataset/nyc-jobs.csv")



# Remove duplicate rows
print("count before remove duplicate",data_file.count())
#data_file = data_file.dropDuplicates()
# Print the schema to understand column types
print("count after removed duplicate",data_file.count())
# Print the schema to understand column types
data_file.printSchema()
# Show the first few rows of the DataFrame
#data_file.show(5)

INFO:py4j.clientserver:Error while sending or receiving.
Traceback (most recent call last):
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/py4j/clientserver.py", line 503, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [Errno 104] Connection reset by peer
INFO:py4j.clientserver:Closing down clientserver connection
INFO:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/py4j/clientserver.py", line 503, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/

count before remove duplicate 2946
count after removed duplicate 2946
root
 |-- Job ID: integer (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Posting Type: string (nullable = true)
 |-- # Of Positions: integer (nullable = true)
 |-- Business Title: string (nullable = true)
 |-- Civil Service Title: string (nullable = true)
 |-- Title Code No: string (nullable = true)
 |-- Level: string (nullable = true)
 |-- Job Category: string (nullable = true)
 |-- Full-Time/Part-Time indicator: string (nullable = true)
 |-- Salary Range From: double (nullable = true)
 |-- Salary Range To: double (nullable = true)
 |-- Salary Frequency: string (nullable = true)
 |-- Work Location: string (nullable = true)
 |-- Division/Work Unit: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- Minimum Qual Requirements: string (nullable = true)
 |-- Preferred Skills: string (nullable = true)
 |-- Additional Information: string (nullable = true)
 |-- To Apply: string (nulla

In [20]:

# Summary statistics for numerical columns
data_file.describe(["# Of Positions", "Salary Range From", "Salary Range To"]).show()

# Count distinct values for categorical columns
categorical_columns = ["Agency", "Posting Type", "Title Code No", "Level", "Job Category", "Full-Time/Part-Time indicator", "Salary Frequency"]

for column in categorical_columns:
    print(f"Distinct values in {column}:")
    data_file.select(column).distinct().show(10, truncate=False)


# Count null values in each column
data_file.select([sum(col(c).isNull().cast("int")).alias(c) for c in data_file.columns]).show()

+-------+------------------+------------------+-----------------+
|summary|    # Of Positions| Salary Range From|  Salary Range To|
+-------+------------------+------------------+-----------------+
|  count|              2946|              2946|             2946|
|   mean|2.4959266802443993| 58904.13979385609|85535.71162739306|
| stddev| 9.281312826466838|26986.575935791363|42871.31345366745|
|    min|                 1|               0.0|            10.36|
|    max|               200|          218587.0|         234402.0|
+-------+------------------+------------------+-----------------+

Distinct values in Agency:


+------------------------------+
|Agency                        |
+------------------------------+
|OFFICE OF COLLECTIVE BARGAININ|
|FIRE DEPARTMENT               |
|ADMIN FOR CHILDREN'S SVCS     |
|MANHATTAN COMMUNITY BOARD #8  |
|TAXI & LIMOUSINE COMMISSION   |
|DEPARTMENT OF BUSINESS SERV.  |
|DEPT OF DESIGN & CONSTRUCTION |
|FINANCIAL INFO SVCS AGENCY    |
|DEPARTMENT OF CORRECTION      |
|HOUSING PRESERVATION & DVLPMNT|
+------------------------------+
only showing top 10 rows

Distinct values in Posting Type:
+------------+
|Posting Type|
+------------+
|Internal    |
|External    |
+------------+

Distinct values in Title Code No:
+-------------+
|Title Code No|
+-------------+
|70822        |
|52408        |
|51193        |
|22508        |
|34615        |
|91001        |
|52040        |
|12158        |
|51011        |
|20415        |
+-------------+
only showing top 10 rows

Distinct values in Level:
+-----+
|Level|
+-----+
|3    |
|M4   |
|M7   |
|4B   |
|0    |
|M6   |
|M1   

#List of KPIs to be resolved

In [21]:
# 1. Count the number of jobs per agency
agency_counts = data_file.groupBy("Agency").count().orderBy("count", ascending=False)
agency_counts.show(10, truncate=False)

+------------------------------+-----+
|Agency                        |count|
+------------------------------+-----+
|DEPT OF ENVIRONMENT PROTECTION|655  |
|NYC HOUSING AUTHORITY         |231  |
|DEPT OF HEALTH/MENTAL HYGIENE |188  |
|DEPARTMENT OF TRANSPORTATION  |183  |
|DEPT OF DESIGN & CONSTRUCTION |142  |
|TAXI & LIMOUSINE COMMISSION   |134  |
|ADMIN FOR CHILDREN'S SVCS     |108  |
|DEPT OF INFO TECH & TELECOMM  |107  |
|LAW DEPARTMENT                |95   |
|HOUSING PRESERVATION & DVLPMNT|86   |
+------------------------------+-----+
only showing top 10 rows



In [22]:
# 2.number of jobs posting per category top 10:
job_category_counts = data_file.filter(col("Job Category").isNotNull()).groupBy("Job Category").count().orderBy("count", ascending=False)
job_category_counts.show(10, truncate=False)

+-----------------------------------------+-----+
|Job Category                             |count|
+-----------------------------------------+-----+
|Engineering, Architecture, & Planning    |504  |
|Technology, Data & Innovation            |313  |
|Legal Affairs                            |226  |
|Public Safety, Inspections, & Enforcement|182  |
|Building Operations & Maintenance        |181  |
|Finance, Accounting, & Procurement       |169  |
|Administration & Human Resources         |134  |
|Constituent Services & Community Programs|129  |
|Health                                   |125  |
|Policy, Research & Analysis              |124  |
+-----------------------------------------+-----+
only showing top 10 rows



In [23]:
# 3. salary distribution per job category
salary_distribution = data_file.groupBy("Job Category").agg(sum("Salary Range From").alias("Total Salary Range From"),
                                                            sum("Salary Range To").alias("Total Salary Range To"),
                                                            sum("# Of Positions").alias("Total Positions")) \
    .orderBy("Total Salary Range From", ascending=False)
salary_distribution.show(10, truncate=False)  




+-----------------------------------------+-----------------------+---------------------+---------------+
|Job Category                             |Total Salary Range From|Total Salary Range To|Total Positions|
+-----------------------------------------+-----------------------+---------------------+---------------+
|Engineering, Architecture, & Planning    |3.3320681810000002E7   |5.188658181E7        |762            |
|Technology, Data & Innovation            |2.1777043960500002E7   |3.33524432098E7      |405            |
|Legal Affairs                            |1.58365392558E7        |2.17911463106E7      |515            |
|Finance, Accounting, & Procurement       |1.00772180668E7        |1.48655259762E7      |275            |
|Public Safety, Inspections, & Enforcement|9363633.7292           |1.32629547906E7      |1407           |
|Policy, Research & Analysis              |6804147.8412           |8963123.9008         |200            |
|Constituent Services & Community Programs|646

In [24]:
# 4.Correlation between the higher degree and the salary.

data_with_degree = data_file.withColumn(
    "Degree Type",
    regexp_extract(col("Minimum Qual Requirements"), r"(\w+)\sdegree", 1)
)

# Show the results
data_with_degree.select("Degree Type").show(10, truncate=False)

+-------------+
|Degree Type  |
+-------------+
|baccalaureate|
|baccalaureate|
|             |
|             |
|             |
|             |
|baccalaureate|
|baccalaureate|
|s            |
|             |
+-------------+
only showing top 10 rows



In [25]:
# 5. Job posting having the highest salary per agency
highest_salary_per_agency = data_file.groupBy("Agency").agg(
    {"Salary Range From": "max", "Salary Range To": "max"}
).orderBy("max(Salary Range To)", ascending=False)
highest_salary_per_agency.show(10, truncate=False)

+------------------------------+----------------------+--------------------+
|Agency                        |max(Salary Range From)|max(Salary Range To)|
+------------------------------+----------------------+--------------------+
|NYC EMPLOYEES RETIREMENT SYS  |130000.0              |234402.0            |
|POLICE DEPARTMENT             |200000.0              |234402.0            |
|NYC HOUSING AUTHORITY         |175000.0              |234402.0            |
|DEPT OF HEALTH/MENTAL HYGIENE |157725.0              |225217.0            |
|DEPT OF ENVIRONMENT PROTECTION|218587.0              |218587.0            |
|DEPT OF DESIGN & CONSTRUCTION |86346.0               |217244.0            |
|DISTRICT ATTORNEY KINGS COUNTY|175000.0              |208826.0            |
|DEPARTMENT OF SANITATION      |87490.0               |202744.0            |
|DEPT OF INFO TECH & TELECOMM  |96020.0               |189000.0            |
|DEPARTMENT OF PROBATION       |79620.0               |180000.0            |

In [26]:
# 6. Job positings average salary per agency for the last 2 years

#finding most lasted year to find last 2 years data.
 
find_year = data_file.withColumn("Year", year(col("Posting Date"))).distinct()
current_year=find_year.filter(col("year").isNotNull()).select("Year").orderBy("Year",ascending=False).first()["Year"]


# Filter data for the last 2 years
filtered_data = data_file.filter(
    (col("Posting Date").isNotNull()) & 
    (year(col("Posting Date")) >= (current_year - 2))
)

# Calculate average salary per agency
average_salary_per_agency = filtered_data.filter(col("Posting Date").isNotNull()) \
    .groupBy("Agency","Posting Date") \
    .agg({"Salary Range From": "avg", "Salary Range To": "avg"}) \
    .orderBy("avg(Salary Range To)", ascending=False)
average_salary_per_agency.show(10, truncate=False)

+------------------------------+-----------------------+----------------------+--------------------+
|Agency                        |Posting Date           |avg(Salary Range From)|avg(Salary Range To)|
+------------------------------+-----------------------+----------------------+--------------------+
|NYC HOUSING AUTHORITY         |2019-05-14T00:00:00.000|103620.0              |234402.0            |
|POLICE DEPARTMENT             |2019-10-30T00:00:00.000|200000.0              |234402.0            |
|DEPT OF ENVIRONMENT PROTECTION|2019-05-28T00:00:00.000|218587.0              |218587.0            |
|DEPT OF DESIGN & CONSTRUCTION |2019-10-11T00:00:00.000|86346.0               |217244.0            |
|NYC HOUSING AUTHORITY         |2019-01-29T00:00:00.000|78574.0               |202744.0            |
|DEPT OF ENVIRONMENT PROTECTION|2017-03-02T00:00:00.000|75338.0               |194395.0            |
|DEPT OF ENVIRONMENT PROTECTION|2017-04-06T00:00:00.000|75338.0               |194395.0    

In [27]:
# 7 highest paid skills in the US market

def find_highest_paid_skills(df: DataFrame) -> DataFrame:
    # Extract skills or keywords from the "Job Description" or "Minimum Qual Requirements" column
    df_with_skills = df.withColumn(
        "Skill",
        regexp_extract(col("Minimum Qual Requirements"), r"(\w+)\s(skill|experience|knowledge)", 1)
    )
    
    # Filter rows where skills are identified
    df_with_skills = df_with_skills.filter(col("Skill").isNotNull())
    
    # Calculate the average salary for each skill
    skill_salary = df_with_skills.groupBy("Skill").agg(
        avg("Salary Range From").alias("Avg Salary Range From"),
        avg("Salary Range To").alias("Avg Salary Range To")
    ).orderBy("Avg Salary Range To", ascending=False)
    
    return skill_salary

# Example usage
highest_paid_skills = find_highest_paid_skills(data_file)
highest_paid_skills.show(10, truncate=False)


+------------+---------------------+-------------------+
|Skill       |Avg Salary Range From|Avg Salary Range To|
+------------+---------------------+-------------------+
|making      |200000.0             |234402.0           |
|responsible |82680.0              |188559.25          |
|research    |69940.0              |180000.0           |
|consultative|130000.0             |137410.0           |
|recent      |69940.0              |133327.0           |
|auditing    |95834.0              |130865.75          |
|fulltime    |70274.18181818182    |126326.0           |
|related     |84767.66666666667    |119502.44444444444 |
|technology  |86932.53125          |119117.5625        |
|specialized |87909.45454545454    |117739.18181818182 |
+------------+---------------------+-------------------+
only showing top 10 rows



In [28]:
try:
    def clean_dataset(df: DataFrame) -> DataFrame:
        print(f"Count before removing duplicates: {df.count()}")
        df = df.dropDuplicates()
        print(f"Count after removing duplicates: {df.count()}")
        return df

    # Function to preprocess columns (e.g., extract year, clean text)
    def preprocess_columns(df: DataFrame) -> DataFrame:
        df = df.withColumn("Year", year(col("Posting Date")))
        df = df.withColumn(
            "Degree Type",
            regexp_extract(col("Minimum Qual Requirements"), r"(\w+)\sdegree", 1)
        )
        return df

    # Function for data wrangling (e.g., filtering, grouping)
    def wrangle_data(df: DataFrame, current_year: int) -> DataFrame:
        # Filter data for the last 2 years
        filtered_data = df.filter(
            (col("Posting Date").isNotNull()) & 
            (year(col("Posting Date")) >= (current_year - 2))
        )
        return filtered_data

    # Function for data transformation (e.g., aggregations, calculations)
    def transform_data(df: DataFrame) -> DataFrame:
        # Example: Calculate average salary per agency
        transformed_df = df.groupBy("Agency").agg(
            sum("Salary Range From").alias("Total Salary Range From"),
            sum("Salary Range To").alias("Total Salary Range To"),
            sum("# Of Positions").alias("Total Positions")
        ).orderBy("Total Salary Range From", ascending=False)
        return transformed_df

    # Clean the dataset
    data_file = clean_dataset(data_file)

    # Preprocess columns
    data_file = preprocess_columns(data_file)

    # Find the current year
    current_year = data_file.select(year(col("Posting Date")).alias("Year")).distinct().orderBy(col("Year").desc()).first()["Year"]

    # Wrangle the data
    filtered_data = wrangle_data(data_file, current_year)

    # Transform the data
    final_data = transform_data(filtered_data)

    # Final transformed data to save as file.
    final_data.write.format("csv").mode("overwrite").option("header", "true").save("/workspaces/data_engineering_task/dataset/final_transformed_data.csv")

    # final data to write to hive table (make sure final table exists).
    spark.sql("CREATE DATABASE IF NOT EXISTS sample_db")
    final_data.write.format("orc").mode("overwrite").saveAsTable("sample_db.final_transformed_data")

except Exception as e:
    logging.error(f"An error occurred: {e}")
finally:
    # Stop the Spark session
    spark.stop()



Count before removing duplicates: 2946
Count after removing duplicates: 2915


25/04/16 17:05:35 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/04/16 17:05:35 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/04/16 17:05:36 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/04/16 17:05:36 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore codespace@127.0.0.1
25/04/16 17:05:37 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
25/04/16 17:05:38 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
25/04/16 17:05:38 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
25/04/16 17:05:38 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/04/16 17:05:38 